In [ ]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# Load model architecture from JSON file
with open("fer2.json", "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

# Load weights into the model
model.load_weights('model_filter.h5')

face_haar_cascade = cv2.CascadeClassifier('haarcascades_haarcascade_frontalface_default.xml')

capture = cv2.VideoCapture(0)

while True:
    # Capture frame and return boolean value and captured image
    ret, frame = capture.read()
    if not ret:
        continue
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), thickness=2)
        # Cropping region of interest, i.e., the face area from the image
        roi_gray = gray_img[y:y+w, x:x+h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        # Find the emotion with maximum confidence
        max_index = np.argmax(predictions[0])

        emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        predicted_emotion = emotions[max_index]

        cv2.putText(frame, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    resized_img = cv2.resize(frame, (1000, 700))
    cv2.imshow('Facial emotion analysis', resized_img)

    if cv2.waitKey(10) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


In [ ]:
# Load model architecture from JSON file
with open("fer2.json", "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)

# Load weights into the model
model.load_weights('model_filter.h5')

# Access the layers and their weights
for layer in model.layers:
    layer_name = layer.name
    layer_weights = layer.get_weights()
    print("Layer Name:", layer_name)
    print("Layer Weights:", layer_weights)
    print()
    


In [ ]:
model.summary()

In [ ]:
import h5py

def explore_hdf5(file, prefix='', max_depth=3, current_depth=0):
    for key in file.keys():
        path = f'{prefix}/{key}'
        if current_depth >= max_depth:
            print(f'Reached max exploration depth at: {path}')
            continue

        if isinstance(file[key], h5py.Group):
            print(f'Group: {path}')
            explore_hdf5(file[key], prefix=path, max_depth=max_depth, current_depth=current_depth + 1)
        elif isinstance(file[key], h5py.Dataset):
            dataset = file[key]
            print(f'Dataset: {path}')
            print(f'Shape: {dataset.shape}')
            # You can choose whether to print values or not
        else:
            print(f'Unknown type: {path}')

# Open the HDF5 file
file_path = 'model_filter.h5'
with h5py.File(file_path, 'r') as file:
    explore_hdf5(file)
